In [6]:
import pandas as pd
import numpy as np

In [44]:
Perfume_df = pd.read_csv("C:/Users/hmq4/OneDrive/Desktop/Data Science- Misk/chrome/Misk_DSI_Capstone_Project/final_df.csv")

In [45]:
Perfume_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4239 entries, 0 to 4238
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Name              4239 non-null   object
 1   Price             4239 non-null   int64 
 2   Description       4239 non-null   object
 3   Rate              4239 non-null   object
 4   Rating_count      4239 non-null   object
 5   image             4239 non-null   object
 6   Brand             4239 non-null   object
 7   Gender            4239 non-null   object
 8   Product_Type      4239 non-null   object
 9   Character_x       4239 non-null   object
 10  Fragrance_Family  4239 non-null   object
 11  Size              4239 non-null   object
 12  Year              4239 non-null   int64 
 13  Ingredients       4239 non-null   object
 14  Concentration     4239 non-null   object
 15  Top_note          4239 non-null   object
 16  Middle_note       4239 non-null   object
 17  Base_note     

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer
#Define a TF-IDF Object. use stop_words to remove english stop words such as 'the', 'a'
tfi = TfidfVectorizer(stop_words='english')
# specify the column I will use Description column and fill in NaNs with a space 
Perfume_df["Description"] = Perfume_df['Description'].fillna(" ")
#create TF-IDF matrix by fitting and transforming the data
tfi_matrix = tfi.fit_transform(Perfume_df['Description'])
#How large the shape of tfidf_matrix
tfi_matrix.shape

(4239, 7485)

In [47]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel
# find the cosine similarity matrix
cosine_sim = linear_kernel(tfi_matrix, tfi_matrix)

In [48]:
# get the perfume index from dataframe 
indices = pd.Series(Perfume_df.index, index=Perfume_df['Name']).drop_duplicates()

In [52]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the perfume that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all perfume with that perfume
    sim_scores = list(enumerate(cosine_sim[idx]))
        
    # Sort the Perfumes based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar perfumes
    sim_scores = sim_scores[1:6]

    # Get the perfume indices
    per_indices = [i[0] for i in sim_scores]
    
    # Return the top 10 most similar perfumes based on description
    return Perfume_df['Name'].iloc[per_indices]

In [53]:
get_recommendations("Dolce & Gabanna L'imperatrice 3 Pour Femme")

104      Ya Aali Al Hima Set
1111                Ebar Set
879     Together Forever Set
1109        Men Mithlak? Set
570        Sokar Aleyoon Set
Name: Name, dtype: object

In [54]:
get_recommendations("Guerlain Santal Royal Guerlain")

164                                Guerlain Oud Essentiel
1088                                 Lootah Silky Oud EDP
862           Salvatore Ferragamo Emozione Rosa Orientale
755     Narciso Rodriguez Santal Musc Eau De Parfum In...
679                   Oscar De La Renta Oscar Velvet Noir
Name: Name, dtype: object

In [55]:
get_recommendations("Tom Ford Noir Extreme")

1211    Tom Ford Noir Pour Femme
3092           Tom Ford Noir EDP
291        Tom Ford Noir De Noir
3022         Tom Ford Metallique
3038       Tom Ford Rive D'ambre
Name: Name, dtype: object

### Using Ingredients to find similar perfumes 

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfi = TfidfVectorizer(stop_words='english')
Perfume_df["Ingredients"] = Perfume_df['Ingredients'].fillna(" ")
tfi_matrix2 = tfi.fit_transform(Perfume_df['Ingredients'])
#shape of tfi_matrix2
tfi_matrix2.shape

(4239, 1514)

The matrix size is much smaller that when we use description, it may indicate that will result in a poor recommendations

In [58]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfi_matrix2, tfi_matrix2)

In [59]:
indices = pd.Series(Perfume_df.index, index=Perfume_df['Name']).drop_duplicates()

In [60]:
def get_recommendations2(title, cosine_sim=cosine_sim):
    # Get the index of the perfume that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all perfume with that perfume
    sim_scores = list(enumerate(cosine_sim[idx]))
        
    # Sort the Perfumes based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar perfumes
    sim_scores = sim_scores[1:6]

    # Get the perfume indices
    per_indices = [i[0] for i in sim_scores]
    
    # Return the top 10 most similar perfumes based on description
    return Perfume_df['Name'].iloc[per_indices]

In [61]:
get_recommendations2("Dolce & Gabbana Pour Femme Intense")

710                   Dolce & Gabbana Pour Femme
79            Candy Addict Marshmallow Mash EDP 
547     Candy Addict Marshmallow Mash Hair Mist 
548     Candy Addict Marshmallow Mash Body Mist 
3000     Victoria's Secret Purple Haze Body Mist
Name: Name, dtype: object

In this I got a wrong recommendations the ingredients of the recommended perfumes are totally different, so I will use other approach in Model_1.ipynb.